# Generating counterfactuals for multi-class classification and regression models
This notebook will demonstrate how the DiCE library can be used for multiclass classification and regression for scikit-learn models.
You can use any method ("random", "kdtree", "genetic"), just specific it in the method argument in the initialization step. The rest of the code is completely identical.
For demonstration, we will be using the genetic algorithm for CFs.

In [195]:
import dice_ml
from dice_ml import Dice

from sklearn.datasets import load_iris, load_boston
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import pandas as pd

In [196]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We will use sklearn's internal datasets to demonstrate DiCE's features in this notebook

## Multiclass Classification

For multiclass classification, we will use sklearn's Iris dataset. This data set consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length. More information at https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset

In [197]:
df_iris = load_iris(as_frame=True).frame
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [198]:
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [199]:
outcome_name = "target"
continuous_features_iris = df_iris.drop(outcome_name, axis=1).columns.tolist()
target = df_iris[outcome_name]

In [221]:
# Split data into train and test
datasetX = df_iris.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=target)

categorical_features = x_train.columns.difference(continuous_features_iris)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_iris),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf_iris = Pipeline(steps=[('preprocessor', transformations),
                           ('classifier', RandomForestClassifier())])
model_iris = clf_iris.fit(x_train, y_train)

In [222]:
continuous_features_iris

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [203]:
import joblib

In [205]:
joblib.dump(model_iris, 'iris_model.pickle')

['iris_model.pickle']

In [206]:
x_train[outcome_name] = y_train

<ipython-input-206-b95ecacab499>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[outcome_name] = y_train


In [213]:
x_train.to_csv('iris_train.csv', index=False)

In [214]:
x_test[0:3].to_csv('iris_query.csv', index=False)

In [212]:
x_test

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
36,5.5,3.5,1.3,0.2
55,5.7,2.8,4.5,1.3
44,5.1,3.8,1.9,0.4
134,6.1,2.6,5.6,1.4
39,5.1,3.4,1.5,0.2
90,5.5,2.6,4.4,1.2
147,6.5,3.0,5.2,2.0
47,4.6,3.2,1.4,0.2
3,4.6,3.1,1.5,0.2
81,5.5,2.4,3.7,1.0


In [223]:
d_iris = dice_ml.Data(dataframe=df_iris,
                      continuous_features=continuous_features_iris,
                      outcome_name=outcome_name)

# We provide the type of model as a parameter (model_type)
m_iris = dice_ml.Model(model=model_iris, backend="sklearn", model_type='classifier')

In [224]:
exp_genetic_iris = Dice(d_iris, m_iris, method="genetic")

As we can see below, all the target values will lie in the desired class

In [229]:
# Single input
query_instances_iris = x_train[2:3]
genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=7, desired_class=2)
genetic_iris.visualize_as_dataframe(show_only_changes = True)

100%|██████████| 1/1 [00:00<00:00,  2.97it/s]

Query instance (original outcome : 0)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.4,3.0,1.3,0.2,0



Diverse Counterfactual set (new outcome: 2.0)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.3,-,5.1,1.7,2.0
0,4.9,-,4.5,1.7,2.0
0,5.7,-,5.1,1.7,2.0
0,5.9,-,4.8,1.8,2.0
0,5.9,-,4.9,1.8,2.0
0,6.0,-,4.8,1.8,2.0
0,5.9,-,5.1,1.8,2.0


In [233]:
genetic_iris = exp_genetic_iris.feature_importance(query_instances_iris, total_CFs=10, desired_class=2)

100%|██████████| 1/1 [00:00<00:00,  2.26it/s]


In [232]:
genetic_iris.cf_examples_list

In [234]:
genetic_iris.summary_importance

{'sepal length (cm)': 1.0,
 'petal length (cm)': 1.0,
 'petal width (cm)': 1.0,
 'sepal width (cm)': 0.3}

In [228]:
# Multiple queries can be given as input at once
query_instances_iris = x_train[17:19]
genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=7, desired_class=2)
genetic_iris.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 2/2 [00:00<00:00,  3.15it/s]

Query instance (original outcome : 1)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.7,2.9,4.2,1.3,1



Diverse Counterfactual set (new outcome: 2.0)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,6.3,2.8,5.1,1.5,2.0
0,6.0,3.0,4.8,1.8,2.0
0,5.9,3.0,5.1,1.8,2.0
0,6.0,3.0,4.9,1.8,2.0
0,6.1,3.0,4.9,1.8,2.0
0,5.6,2.8,4.9,2.0,2.0
0,6.2,2.8,4.8,1.8,2.0


Query instance (original outcome : 1)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,6.3,3.3,4.7,1.6,1



Diverse Counterfactual set (new outcome: 2.0)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,7.8,-,3.1,1.9,2.0
0,6.2,3.4,5.1,1.5,2.0
0,6.2,3.4,4.9,2.0,2.0
0,6.5,3.2,5.1,2.0,2.0
0,6.4,3.1,5.5,1.8,2.0
0,6.1,3.0,4.9,1.8,2.0
0,6.0,3.0,4.8,1.8,2.0


# Regression

For regression, we will use sklearn's boston dataset. This dataset contains boston house-prices. More information at https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-house-prices-dataset

In [12]:
boston_data = load_boston()
df_boston = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
df_boston[outcome_name] = pd.Series(boston_data.target)
df_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [13]:
df_boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  target   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [14]:
continuous_features_boston = df_boston.drop(outcome_name, axis=1).columns.tolist()
target = df_boston[outcome_name]

In [15]:
# Split data into train and test
datasetX = df_boston.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0)

categorical_features = x_train.columns.difference(continuous_features_boston)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_boston),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
regr_boston = Pipeline(steps=[('preprocessor', transformations),
                              ('regressor', RandomForestRegressor())])
model_boston = regr_boston.fit(x_train, y_train)

In [16]:
d_boston = dice_ml.Data(dataframe=df_boston, continuous_features=continuous_features_boston, outcome_name=outcome_name)
# We provide the type of model as a parameter (model_type)
m_boston = dice_ml.Model(model=model_boston, backend="sklearn", model_type='regressor')

In [181]:
exp_genetic_boston = Dice(d_boston, m_boston, method="random")

As we can see below, all the target values will lie in the desired range

In [182]:
# Multiple queries can be given as input at once
query_instances_boston = x_train[2:3]
genetic_boston = exp_genetic_boston.generate_counterfactuals(query_instances_boston,
                                                             total_CFs=2,
                                                             desired_range=[30, 45])
genetic_boston.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  3.92it/s]

Query instance (original outcome : 24)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.11329,30.0,4.93,0.0,0.428,6.897,54.3,6.3361,6.0,300.0,16.6,391.25,11.38,24.0



Diverse Counterfactual set (new outcome: [30, 45])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,-,-,-,-,-,7.966,-,-,-,-,-,-,-,35.96099853515625
1,-,-,-,0.3,-,-,-,-,-,-,-,-,4.6,30.92300033569336


In [183]:
exp_genetic_boston.generate_counterfactuals?

In [184]:
# Multiple queries can be given as input at once
query_instances_boston = x_train[17:19]
genetic_boston = exp_genetic_boston.feature_importance(query_instances_boston, 
                                                             total_CFs=30, 
                                                             desired_range=[40, 50],
                                                            )
genetic_boston.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 2/2 [00:01<00:00,  1.05it/s]

Query instance (original outcome : 49)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.01501,90.0,1.21,1.0,0.401,7.923,24.8,5.885,1.0,198.0,13.6,395.52,3.16,49.0



Diverse Counterfactual set (new outcome: [40, 50])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,-,-,-,-,-,-,86.2,-,-,-,-,-,-,48.20000076293945
1,-,-,23.7,-,-,-,-,8.3229,-,-,-,-,-,48.064998626708984
2,1.41168,-,-,-,-,-,-,-,-,-,-,-,-,48.94599914550781
3,79.37284,-,-,-,-,-,22.4,-,-,-,-,-,-,47.275001525878906
4,-,-,-,-,-,-,98.2,-,15.1,-,-,-,-,45.744998931884766
5,-,62.1,-,-,-,-,-,-,-,592.8,-,-,-,48.45399856567383
6,-,-,-,-,0.75,-,-,-,-,-,-,-,-,48.29999923706055
7,-,-,-,-,-,-,-,-,11.7,371.8,-,-,-,46.72800064086914
8,-,-,-,-,0.806,-,-,-,-,538.6,-,-,-,47.9640007019043
9,-,-,15.7,-,-,-,-,-,18.5,-,-,-,-,45.99800109863281


Query instance (original outcome : 30)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.06911,45.0,3.44,0.0,0.437,6.739,30.8,6.4798,5.0,398.0,15.2,389.71,4.69,30.0



Diverse Counterfactual set (new outcome: [40, 50])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,-,-,-,-,-,8.31,-,-,-,-,-,-,-,46.513999938964844
1,-,-,-,-,-,8.65,-,-,-,-,-,-,-,46.50199890136719
2,-,20.6,-,-,-,8.552,-,-,-,-,-,-,-,46.47999954223633
3,-,-,-,-,-,8.725,-,-,-,-,12.8,-,-,47.417999267578125
4,-,-,-,-,-,8.042,-,-,-,-,16.9,-,-,45.16600036621094
5,-,-,-,0.5,-,7.699,-,-,-,-,-,-,-,43.08100128173828
6,-,-,2.7,-,-,8.135,-,-,-,-,-,-,-,46.63399887084961
7,-,-,10.8,-,-,8.002,-,-,-,-,-,-,-,46.24700164794922
8,-,-,-,-,-,8.042,-,-,-,-,-,-,-,46.37300109863281
9,-,-,-,-,-,8.466,-,-,-,-,-,-,-,46.50199890136719


In [185]:
tabu_changes = {}
tabu_changes['AGE'] = [lambda delta: delta>0]

In [186]:
from dice_ml.filter_interfaces import filtering

F = filtering.FilterChanges(tabu_changes)

In [187]:
new_cf_examples_list = F.filtered_cf_list(genetic_boston.cf_examples_list)

Number of excluded counterfactuals is 5


In [188]:
for new_list in new_cf_examples_list:
    print(new_list.final_cfs_df)
    print('-----')

        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE      DIS   RAD    TAX  \
0    0.01501  90.0  23.70   1.0  0.401  7.923  24.8   8.3229   1.0  198.0   
1    1.41168  90.0   1.21   1.0  0.401  7.923  24.8   5.8850   1.0  198.0   
2   79.37284  90.0   1.21   1.0  0.401  7.923  22.4   5.8850   1.0  198.0   
3    0.01501  62.1   1.21   1.0  0.401  7.923  24.8   5.8850   1.0  592.8   
4    0.01501  90.0   1.21   1.0  0.750  7.923  24.8   5.8850   1.0  198.0   
5    0.01501  90.0   1.21   1.0  0.401  7.923  24.8   5.8850  11.7  371.8   
6    0.01501  90.0   1.21   1.0  0.806  7.923  24.8   5.8850   1.0  538.6   
7    0.01501  90.0  15.70   1.0  0.401  7.923  24.8   5.8850  18.5  198.0   
8    0.01501  90.0   1.21   0.9  0.401  7.923  24.8   5.8850   1.0  198.0   
9    0.01501  90.0   1.21   1.0  0.401  7.923   4.7   5.8850   1.0  198.0   
10   3.46989  90.0   1.21   1.0  0.401  7.923  24.8   5.8850   1.0  198.0   
11  14.18258  90.0  12.70   1.0  0.401  7.923  24.8   5.8850   1.0  198.0   

In [189]:
genetic_boston.local_importance

[{'AGE': 0.23333333333333334,
  'DIS': 0.2,
  'RAD': 0.2,
  'TAX': 0.16666666666666666,
  'CRIM': 0.13333333333333333,
  'NOX': 0.13333333333333333,
  'B': 0.13333333333333333,
  'ZN': 0.1,
  'INDUS': 0.1,
  'CHAS': 0.1,
  'PTRATIO': 0.06666666666666667,
  'LSTAT': 0.03333333333333333,
  'RM': 0.0},
 {'RM': 1.0,
  'RAD': 0.13333333333333333,
  'PTRATIO': 0.13333333333333333,
  'INDUS': 0.1,
  'TAX': 0.1,
  'CRIM': 0.03333333333333333,
  'ZN': 0.03333333333333333,
  'CHAS': 0.03333333333333333,
  'LSTAT': 0.03333333333333333,
  'NOX': 0.0,
  'AGE': 0.0,
  'DIS': 0.0,
  'B': 0.0}]

In [190]:
genetic_boston = exp_genetic_boston.feature_importance(query_instances_boston, 
                                                       cf_examples_list = new_cf_examples_list)
genetic_boston.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 49)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.01501,90.0,1.21,1.0,0.401,7.923,24.8,5.885,1.0,198.0,13.6,395.52,3.16,49.0



Diverse Counterfactual set (new outcome: [40, 50])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,-,-,23.7,-,-,-,-,8.3229,-,-,-,-,-,48.0
1,1.41168,-,-,-,-,-,-,-,-,-,-,-,-,-
2,79.37284,-,-,-,-,-,22.4,-,-,-,-,-,-,47.0
3,-,62.1,-,-,-,-,-,-,-,592.8,-,-,-,48.0
4,-,-,-,-,0.75,-,-,-,-,-,-,-,-,48.0
5,-,-,-,-,-,-,-,-,11.7,371.8,-,-,-,47.0
6,-,-,-,-,0.806,-,-,-,-,538.6,-,-,-,48.0
7,-,-,15.7,-,-,-,-,-,18.5,-,-,-,-,46.0
8,-,-,-,0.9,-,-,-,-,-,-,14.2,-,-,-
9,-,-,-,-,-,-,4.7,-,-,-,-,-,-,-


Query instance (original outcome : 30)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.06911,45.0,3.44,0.0,0.437,6.739,30.8,6.4798,5.0,398.0,15.2,389.71,4.69,30.0



Diverse Counterfactual set (new outcome: [40, 50])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,-,-,-,-,-,8.31,-,-,-,-,-,-,-,47.0
1,-,-,-,-,-,8.65,-,-,-,-,-,-,-,47.0
2,-,20.6,-,-,-,8.552,-,-,-,-,-,-,-,46.0
3,-,-,-,-,-,8.725,-,-,-,-,12.8,-,-,47.0
4,-,-,-,-,-,8.042,-,-,-,-,16.9,-,-,45.0
5,-,-,-,0.5,-,7.699,-,-,-,-,-,-,-,43.0
6,-,-,2.7,-,-,8.135,-,-,-,-,-,-,-,47.0
7,-,-,10.8,-,-,8.002,-,-,-,-,-,-,-,46.0
8,-,-,-,-,-,8.042,-,-,-,-,-,-,-,46.0
9,-,-,-,-,-,8.466,-,-,-,-,-,-,-,47.0


In [191]:
genetic_boston.local_importance

[{'DIS': 0.2,
  'RAD': 0.2,
  'TAX': 0.2,
  'CRIM': 0.16,
  'NOX': 0.16,
  'B': 0.16,
  'INDUS': 0.12,
  'CHAS': 0.12,
  'ZN': 0.08,
  'AGE': 0.08,
  'PTRATIO': 0.08,
  'LSTAT': 0.04,
  'RM': 0.0},
 {'RM': 1.0,
  'PTRATIO': 0.16,
  'RAD': 0.12,
  'INDUS': 0.08,
  'CRIM': 0.04,
  'ZN': 0.04,
  'CHAS': 0.04,
  'TAX': 0.04,
  'LSTAT': 0.04,
  'NOX': 0.0,
  'AGE': 0.0,
  'DIS': 0.0,
  'B': 0.0}]

In [192]:
from dice_ml.reason_generator_interfaces import reason_templates, reason_generator

from typing import List

import numpy as np

In [193]:
R = reason_generator.ReasonGenerator(exp_genetic_boston)

In [194]:
R.generate_reasons(new_cf_examples_list, genetic_boston.local_importance, threshold_importance = 0.6)

For query number 0:
No feature with threshold of at least 0.6


Counterfactual number 0 of query 0:
when INDUS increases from 1.210 to 23.700,  and when DIS increases from 5.885 to 8.323,  the output of the model [target] decreases from 49.000 to 48.000.
Counterfactual number 1 of query 0:
when CRIM increases from 0.015 to 1.412,  the output of the model [target] stays from 49.000 to 49.000.
Counterfactual number 2 of query 0:
when CRIM increases from 0.015 to 79.373,  and when AGE decreases from 24.800 to 22.400,  the output of the model [target] decreases from 49.000 to 47.000.
Counterfactual number 3 of query 0:
when ZN decreases from 90.000 to 62.100,  and when TAX increases from 198.000 to 592.800,  the output of the model [target] decreases from 49.000 to 48.000.
Counterfactual number 4 of query 0:
when NOX increases from 0.401 to 0.750,  the output of the model [target] decreases from 49.000 to 48.000.
Counterfactual number 5 of query 0:
when RAD increases from 1.000 to 11.700, 